In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as stats
import warnings
warnings.filterwarnings("ignore")

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("Setup complete.\n")

Setup complete.



In [2]:
df = pd.read_csv("pyrolysis.csv")
units_df = pd.read_csv("pyrolysis_column_units.csv")

print(f"\nLoaded pyrolysis.csv: {df.shape}")
print(f"Loaded pyrolysis_column_units.csv: {units_df.shape}")


Loaded pyrolysis.csv: (751, 17)
Loaded pyrolysis_column_units.csv: (1, 17)


In [3]:
print("\nFirst 5 rows of main dataset:")
display(df.head())

print("\nUnits mapping:")
display(units_df)


First 5 rows of main dataset:


,Index,Biomass species,M,Ash,VM,FC,C,H,O,N,PS,FT,HR,FR,Solid phase,Liquid phase,Gas phase
0,1,Jerusalem artichoke stick,15.76,3.34,67.40,13.50,45.36,6.11,47.26,0.75,0.5,550,10.0,100.0,35,44.9,20.1
1,2,Jerusalem artichoke stick,15.76,3.34,67.40,13.50,45.36,6.11,47.26,0.75,0.5,650,10.0,100.0,31.75,41.25,27
2,3,Jerusalem artichoke stick,15.76,3.34,67.40,13.50,45.36,6.11,47.26,0.75,0.5,750,10.0,100.0,30.2,40.6,29.2
3,4,Jerusalem artichoke stick,15.76,3.34,67.40,13.50,45.36,6.11,47.26,0.75,0.5,850,10.0,100.0,28.6,36.36,35.04
4,5,reed,5.89,8.47,72.12,13.52,42.78,5.17,50.51,1.33,0.5,550,10.0,100.0,32.85,54.38,12.77



Units mapping:


,Index,Biomass species,M,Ash,VM,FC,C,H,O,N,PS,FT,HR,FR,Solid phase,Liquid phase,Gas phase
0,NaN,NaN,%,%,%,%,%,%,%,%,mm,℃,℃,mL/min,%,%,%


In [13]:
print("="*60)
print("AUTO COLUMN DETECTION & CLEANING")
print("="*60)

expected_numeric = [
    'M', 'Ash', 'VM', 'FC', 'C', 'H', 'O', 'N',
    'PS', 'FT', 'HR', 'FR',
    'Solid phase', 'Liquid phase', 'Gas phase'
]

existing_numeric = [col for col in expected_numeric if col in df.columns]
print(f"Found {len(existing_numeric)} numeric columns: {existing_numeric}")

for col in existing_numeric:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(r'[^0-9.-]', '', regex=True)
        .replace('', np.nan)
    )
    df[col] = pd.to_numeric(df[col], errors='coerce')

yield_cols = [c for c in ['Solid phase', 'Liquid phase', 'Gas phase'] if c in df.columns]
df = df.dropna(subset=yield_cols, how='any').copy()

print(f"After cleaning: {df.shape[0]} rows kept")
print(f"Missing values in numeric columns:\n{df[existing_numeric].isna().sum()[lambda x: x>0]}")

AUTO COLUMN DETECTION & CLEANING
Found 14 numeric columns: ['M', 'VM', 'FC', 'C', 'H', 'O', 'N', 'PS', 'FT', 'HR', 'FR', 'Solid phase', 'Liquid phase', 'Gas phase']
After cleaning: 715 rows kept
Missing values in numeric columns:
M      92
VM     68
FC     68
C       3
H       3
O       3
N       3
PS     85
HR     77
FR    145
dtype: int64


In [14]:
print("\n" + "="*60)
print("MASS BALANCE VALIDATION")
print("="*60)

df['Total_Yield'] = df[yield_cols].sum(axis=1)
df['Balance_Error (%)'] = np.abs(df['Total_Yield'] - 100.0)
df['Balance_OK'] = df['Balance_Error (%)'] <= 1.5

ok = df['Balance_OK'].sum()
print(f"Mass balance within ±1.5%: {ok}/{len(df)} rows")

bad = df[~df['Balance_OK']]
if len(bad) > 0:
    print(f"\n{len(bad)} problematic rows (first 5):")
    show_cols = ['Index'] + (['Species'] if 'Species' in df.columns else []) + yield_cols + ['Total_Yield', 'Balance_Error (%)']
    display(bad[show_cols].head())
else:
    print("All rows pass mass balance.")


MASS BALANCE VALIDATION
Mass balance within ±1.5%: 507/715 rows

208 problematic rows (first 5):


,Index,Solid phase,Liquid phase,Gas phase,Total_Yield,Balance_Error (%)
14,15,23.33,29.30,45.80,98.43,1.57
15,16,56.69,25.04,14.80,96.53,3.47
34,35,23.12,34.35,40.84,98.31,1.69
35,36,21.48,33.76,42.10,97.34,2.66
41,42,64.87,13.89,23.30,102.06,2.06


In [15]:
df_eng = df.copy()

for col in ['HR', 'FR']:
    if col in df_eng.columns:
        med = df_eng[col].median()
        df_eng[col] = df_eng[col].fillna(med)
        print(f"Filled {col} NaN → {med:.2f}")

if all(c in df_eng.columns for c in ['C', 'O', 'H']):
    eps = 1e-6
    df_eng['O/C'] = df_eng['O'] / (df_eng['C'] + eps)
    df_eng['H/C'] = df_eng['H'] / (df_eng['C'] + eps)
    print("Added O/C and H/C ratios")
if all(c in df_eng.columns for c in ['VM', 'FC']):
    df_eng['VM/FC'] = df_eng['VM'] / (df_eng['FC'] + eps)
    print("Added VM/FC ratio")


if all(c in df_eng.columns for c in ['FT', 'HR']):
    df_eng['FT_HR'] = df_eng['FT'] * df_eng['HR']
if all(c in df_eng.columns for c in ['PS', 'FT']):
    df_eng['PS_FT'] = df_eng['PS'] * df_eng['FT']

print("Feature engineering complete.\n")

Filled HR NaN → 20.00
Filled FR NaN → 100.00
Added O/C and H/C ratios
Added VM/FC ratio
Feature engineering complete.



In [16]:
print("="*60)
print("EXPLORATORY DATA ANALYSIS")
print("="*60)


num_cols_for_eda = [c for c in existing_numeric if c not in yield_cols]
num_cols_for_eda += [c for c in ['O/C', 'H/C', 'VM/FC', 'FT_HR', 'PS_FT'] if c in df_eng.columns]
print("Numeric summary:")
display(df_eng[num_cols_for_eda].describe().T.round(3))


corr_cols = num_cols_for_eda + yield_cols
corr = df_eng[corr_cols].corr()

fig = go.Figure(go.Heatmap(
    z=corr.values, x=corr.columns, y=corr.columns,
    colorscale='RdBu', zmid=0,
    text=corr.values.round(2), texttemplate="%{text}",
    textfont_size=9
))
fig.update_layout(title="Correlation Matrix", width=900, height=750)
fig.show()


fig_ternary = px.scatter_ternary(
    df_eng,
    a="Solid phase", b="Liquid phase", c="Gas phase",
    color="FT" if "FT" in df_eng.columns else None,
    size="HR" if "HR" in df_eng.columns else None,
    hover_data=["Species"] if "Species" in df_eng.columns else None,
    title="Pyrolysis Yield Ternary Diagram"
)
fig_ternary.update_layout(height=600)
fig_ternary.show()


if "FT" in df_eng.columns:
    fig_yield = make_subplots(rows=1, cols=3, subplot_titles=yield_cols)
    for i, col in enumerate(yield_cols):
        fig_yield.add_trace(go.Scatter(x=df_eng['FT'], y=df_eng[col], mode='markers', name=col), row=1, col=i+1)
    fig_yield.update_layout(height=400, title="Yields vs Final Temperature (°C to C)")
    fig_yield.show()

EXPLORATORY DATA ANALYSIS
Numeric summary:


,count,mean,std,min,25%,50%,75%,max
M,623.0,6.041,5.189,-100.000,4.900,5.700,7.360,22.740
VM,647.0,73.386,10.207,5.820,70.330,74.900,78.890,89.800
FC,647.0,14.735,6.363,0.200,11.300,14.080,18.900,39.340
C,712.0,47.766,6.499,22.490,44.410,47.200,52.250,66.650
H,712.0,6.582,1.745,3.000,5.810,6.190,6.740,21.200
O,712.0,43.398,7.933,24.900,39.100,45.100,48.280,73.680
N,712.0,2.624,3.263,0.000,0.750,1.530,3.900,22.500
PS,630.0,5.174,24.463,0.100,0.430,0.650,1.200,175.000
FT,715.0,512.818,106.376,300.000,450.000,500.000,550.000,900.000
HR,715.0,37.271,66.651,0.000,10.000,20.000,30.000,700.000


In [22]:
print("\n" + "="*60)
print("MACHINE LEARNING: LIQUID PHASE PREDICTION")
print("="*60)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
import lightgbm as lgb

target = 'Liquid phase'
if target not in df_eng.columns:
    print(f"Warning: '{target}' not found. Skipping modeling.")
else:
    num_features = [c for c in df_eng.select_dtypes(include=[np.number]).columns
                    if c not in yield_cols + ['Total_Yield', 'Balance_Error (%)']]
    cat_features = [c for c in ['Biomass', 'Species'] if c in df_eng.columns]

    X = df_eng[num_features + cat_features]
    y = df_eng[target]

    num_pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    cat_pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ohe', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer([
        ('num', num_pipe, num_features),
        ('cat', cat_pipe, cat_features) if cat_features else ('cat', 'passthrough', [])
    ])


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    models = {
        "Linear Regression": LinearRegression(),
        "Random Forest": RandomForestRegressor(
            n_estimators=300, max_depth=8, random_state=42, n_jobs=-1
        ),
        "Gradient Boosting": GradientBoostingRegressor(
            n_estimators=300, max_depth=4, learning_rate=0.05, random_state=42
        ),
        "XGBoost": XGBRegressor(
            n_estimators=400, max_depth=5, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8, random_state=42, n_jobs=-1
        ),
        "LightGBM": lgb.LGBMRegressor(
            n_estimators=400, max_depth=6, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8, random_state=42, n_jobs=-1
        )
    }

    results = []
    for name, model in models.items():
        pipe = Pipeline([('prep', preprocessor), ('model', model)])
        pipe.fit(X_train, y_train)
        pred = pipe.predict(X_test)

        mae = mean_absolute_error(y_test, pred)
        rmse = np.sqrt(mean_squared_error(y_test, pred))
        r2 = r2_score(y_test, pred)

        results.append({
            "Model": name,
            "MAE": round(mae, 3),
            "RMSE": round(rmse, 3),
            "R²": round(r2, 4)
        })
        print(f"{name:20} → MAE: {mae:6.3f} | RMSE: {rmse:6.3f} | R²: {r2:6.4f}")

    results_df = pd.DataFrame(results)
    print("\nMODEL PERFORMANCE SUMMARY:")
    display(results_df.sort_values("MAE"))

    best_model_name = results_df.loc[results_df['MAE'].idxmin(), 'Model']
    print(f"\nBest performing model: {best_model_name}")

print("\n" + "="*70)
print("PROJECT COMPLETE – SUMMARY")
print("="*70)
print(f"Final dataset: {len(df_eng)} rows")
print(f"Mass balance OK: {ok}/{len(df_eng)}")
print(f"Numeric columns used: {len(existing_numeric)}")
if 'results_df' in locals():
    best = results_df.sort_values("MAE").iloc[0]
    print(f"Best model: {best['Model']} → R² = {best['R²']}, MAE = {best['MAE']}%")
print("All plots, models, and tables are ready for publication.")
print("="*70)


MACHINE LEARNING: LIQUID PHASE PREDICTION
Linear Regression    → MAE:  7.395 | RMSE:  9.196 | R²: 0.3845
Random Forest        → MAE:  2.645 | RMSE:  3.763 | R²: 0.8969
Gradient Boosting    → MAE:  2.568 | RMSE:  3.633 | R²: 0.9039
XGBoost              → MAE:  2.412 | RMSE:  3.399 | R²: 0.9159
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000133 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1191
[LightGBM] [Info] Number of data points in the train set: 572, number of used features: 18
[LightGBM] [Info] Start training from score 38.908636
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


,Model,MAE,RMSE,R²
3,XGBoost,2.412,3.399,0.9159
4,LightGBM,2.499,3.516,0.9100
2,Gradient Boosting,2.568,3.633,0.9039
1,Random Forest,2.645,3.763,0.8969
0,Linear Regression,7.395,9.196,0.3845



Best performing model: XGBoost

PROJECT COMPLETE – SUMMARY
Final dataset: 715 rows
Mass balance OK: 507/715
Numeric columns used: 14
Best model: XGBoost → R² = 0.9159, MAE = 2.412%
All plots, models, and tables are ready for publication.
